# DA-DAN Modulation Analysis Project

### Set-Up

In [ ]:
### imports
import navis
import fafbseg
import flybrains

import numpy as np
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from functools import reduce
from tabulate import tabulate
import hvplot.pandas
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import rgb2hex, to_rgb
import matplotlib.gridspec as gridspec

import pyroglancer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer, set_flywireviewerstate

import navis.interfaces.neuprint as neu
from navis.interfaces.neuprint import NeuronCriteria as NC, SynapseCriteria as SC
from navis.interfaces.neuprint import fetch_adjacencies, fetch_synapse_connections

from pyroglancer.layers import create_nglayer, setlayerproperty
from pyroglancer.ngviewer import openviewer, closeviewer,setviewerstate, get_ngscreenshot
from pyroglancer.ngspaces import create_ngspace
from pyroglancer.createconfig import createconfig

from neuprint import fetch_synapses, NeuronCriteria as NC, SynapseCriteria as SC


import re

import utils
from utils import loadConnections, extractOutputsPerType, extractInputsPerType, plotPAMStatistic, loadPickle


In [ ]:
import warnings
from pandas.errors import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
"""
label_fontsize = 16 
title_fontsize = 18

figwidth_size = 5
figheigth_size = 5

#navis display options for frontal view..
elev = -180
azim = -90
dist = 6
"""

In [ ]:
print('navis version :',navis.__version__)
print('fafbseg version :',fafbseg.__version__)
print('flybrains version :',flybrains.__version__)
print('pyroglancer version :',pyroglancer.__version__)


### Fetch data from Neuprint

In [ ]:
## setting up Neuprint Client
## using dotenv to import Janelia PAT
from dotenv import load_dotenv
import os
load_dotenv()

## fetching Janelia client
client = neu.Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.2.1', token=os.environ.get("JANELIA_PAT"))
client

In [ ]:
### print roi hierarchy
rois = neu.fetch_roi_hierarchy(False,True,'text')
print(rois)

## Analysis

#### Get PAM neurons from hemibrains and fetch their connections

In [ ]:
## fetch all neurons containing "PAM" from hemibrain dataset
pamneurons_df, roi_counts_df = neu.fetch_neurons(NC(status='Traced',type="^PAM.*",regex=True)) 

In [ ]:
### print unique PAM types
uniquePAMTypes = pamneurons_df.type.unique()
print(uniquePAMTypes)

In [ ]:
### get PAM-PAM connections
PAM_PAM_Connections = loadConnections(silent=False)

In [ ]:
### extract unique neuron instances from PAM PAM connections and prepare for dict
uniquePAMNeuronsPre = PAM_PAM_Connections["instance_pre"].unique()
print(uniquePAMNeuronsPre)
uniquePAMNeuronsPost = PAM_PAM_Connections["instance_post"].unique()
print("")
print(uniquePAMNeuronsPost)

# Parse the unique values into a string for a dict
possibleTargetsInstance = "{" + ",\n".join(f"'{neuron}': '{neuron}'" for neuron in sorted(set(uniquePAMNeuronsPre).union(uniquePAMNeuronsPost))) + "}"
print("\nPossible Targets Instance for dict:")
print(possibleTargetsInstance)


#### Heatmap Visualization of PAM-PAM connections

In [ ]:
PAMSuperTypeCollapsedConnections = PAM_PAM_Connections.replace(to_replace=r'PAM(\d{2})\_?\w?', value=r'PAM', regex=True)

#### visualize as matrix
matrix = neu.connection_table_to_matrix(PAMSuperTypeCollapsedConnections, 'type', sort_by='type',)
    ### note: this originally threw an error bc of deprecated call to df.pivot(), fixed it by updating the pivot call in neuprint/utils.py to:
    ### matrix = agg_weights_df.pivot(index=col_pre, columns=col_post, values=weight_col)

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

title="PAM-PAM Connections Heatmap"
matrix.hvplot.heatmap(height=600, width=700, xaxis='top', title=title).opts(xrotation=60)


In [ ]:
#### visualize as matrix
matrix = neu.connection_table_to_matrix(PAM_PAM_Connections, 'type', sort_by='type',)
    ### note: this originally threw an error bc of deprecated call to df.pivot(), fixed it by updating the pivot call in neuprint/utils.py to:
    ### matrix = agg_weights_df.pivot(index=col_pre, columns=col_post, values=weight_col)

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

title="PAM-PAM Connections Heatmap"
matrix.hvplot.heatmap(height=600, width=700, xaxis='top', title=title).opts(xrotation=60)


In [ ]:
### collapse PAM types and then show as matrix

PAMCollapsedConnections = PAM_PAM_Connections.replace(to_replace=r'PAM(\d{2})\_?\w?', value=r'PAM\1', regex=True)

matrix = neu.connection_table_to_matrix(PAMCollapsedConnections, 'type', sort_by='type',)
    ### note: this originally threw an error bc of deprecated call to df.pivot(), fixed it by updating the pivot call in neuprint/utils.py to:
    ### matrix = agg_weights_df.pivot(index=col_pre, columns=col_post, values=weight_col)

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

matrix.hvplot.heatmap(height=600, width=700, xaxis='top', title = title+" -- PAM Types Collapsed").opts(xrotation=60)

In [ ]:
### show unique neurons as matrix
matrix = neu.connection_table_to_matrix(PAM_PAM_Connections, 'instance', sort_by='instance',)
    ### note: this originally threw an error bc of deprecated call to df.pivot(), fixed it by updating the pivot call in neuprint/utils.py to:
    ### matrix = agg_weights_df.pivot(index=col_pre, columns=col_post, values=weight_col)

matrix.index = matrix.index.astype(str)
matrix.columns = matrix.columns.astype(str)

title="PAM-PAM Connections Heatmap"
matrix.hvplot.heatmap(height=600, width=700, xaxis='top', title=title+" -- PAM neurons").opts(xrotation=60)


#### detailed connection analysis

Prints out unique pre- and postsynaptic connection partners (grouped by supertype, type, subtype, instance, and bodyID) of a specific PAM type or all PAMs.

In [ ]:
print("Printing unique connection partners of PAM04 neurons")
subTypePartners, instancePartners, bodyIdPartners = utils.listUniqueConnectionPartners(extractInputsPerType("PAM04",connections=PAM_PAM_Connections))

In [ ]:
### retrieving all connections between PAM neurons and any neurons
PAM_All_Connections = loadConnections("^PAM.*","^.*", bidirectional=True)

In [ ]:
PAM_All_Connections['weight'].plot(kind='hist', bins=201, title='Frequency of Connections by Weight', logy=True)
# Calculate the fifth percentile threshold value for the 'weight' column
fifth_percentile_threshold = PAM_All_Connections['weight'].quantile(0.05)
print(f"Fifth percentile threshold value for connection weights: {fifth_percentile_threshold}")

plt.xlabel('Weight')
plt.ylabel('Frequency')
plt.show()


In [ ]:
print("Printing out unique connection partners of all PAMs")
print("\n Presynaptic")
typePartners, instancePartners, bodyIdPartners = utils.listUniqueConnectionPartners(PAM_All_Connections,type="pre")
print("\n Postsynaptic")
typePartners, instancePartners, bodyIdPartners = utils.listUniqueConnectionPartners(PAM_All_Connections,type="post")

In [ ]:
##testing whether connections make sense
extractInputsPerType(target="PAM05",connections=PAM_All_Connections)
extractOutputsPerType(target="PAM05",connections=PAM_All_Connections)

In [ ]:
### 
targets = ["PAM01","PAM02","PAM03","PAM04"]
partnerType = "type" ## instance, type, bodyId
#connections = filteredPAMConnections
connections = PAM_PAM_Connections
treshhold= 0.005
#plotPAMStatistic(connections=connections,targets=targets,etcTreshhold=treshhold,partnerMode=partnerType,normalized=False, mergePAMSubtypes = True, title="All PAM synapses statistic", mergePAMsupertype=  False, weightFilterThreshhold=1)

In [ ]:
### 
targets = ["PAM01_a(y5)_L"]
partnerType = "type" ## instance, type, bodyId

treshhold= 0.02
#plotPAMStatistic(connections=filteredPAMConnections,targets=targets,targetMode="instance",etcTreshhold=treshhold,partnerMode=partnerType,normalized=False, mergePAMSubtypes = False, title="All PAM synapses statistic", mergePAMs = False)



#### Visualization of PAM-PAM synapses

In [ ]:
# RETRIEVE ALL PAM-PAM SYNAPSES

## get PAM neurons 
neuron_criteria = NC(status='Traced', type="^PAM.*",regex=True)

## get all synapses, within and outside MB
allPAMPAMpresynapses_criteria = SC(type='pre', primary_only=True)
allPAMPAMpostsynapses_criteria = SC(type='post', primary_only=True)
#all_PAM_PAM_presynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, allPAMPAMpresynapses_criteria,batch_size=10000)
#all_PAM_PAM_postsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, allPAMPAMpostsynapses_criteria,batch_size=10000)
### loads cached version of above dataframes
all_PAM_PAM_presynapses = loadPickle('all_PAM_PAM_presynapses')
all_PAM_PAM_postsynapses = loadPickle('all_PAM_PAM_postsynapses')

In [ ]:
from utils import plotSynapseConfidence


plotSynapseConfidence(all_PAM_PAM_presynapses, mode="pre")
plotSynapseConfidence(all_PAM_PAM_postsynapses, mode="post")

In [ ]:
from utils import filterSynapseConfidence

PAMPAMpresynapses = filterSynapseConfidence(all_PAM_PAM_presynapses,mode="pre")
PAMPAMpostsynapses = filterSynapseConfidence(all_PAM_PAM_postsynapses,mode="post")

In [ ]:
#print(rois)
from utils import getAnatomicalOutlines, plotSynapseGroups, plotSynapseClassification

In [ ]:

#fetch 2d outlines of mushroombody (based on neuprint mesh)
MB_outlines_xz = getAnatomicalOutlines("MB")
MB_outlines_xy = getAnatomicalOutlines("MB",'xy')

## fetch alpha lobe
aL_outlines_xz = getAnatomicalOutlines("aL")
aL_outlines_xy = getAnatomicalOutlines("aL",'xy')

In [ ]:
MB_outlines_xz

In [ ]:
title = "PAM-PAM synapses (red: pre, blue: post)"
plotSynapseGroups([PAMPAMpresynapses,PAMPAMpostsynapses],title,ROIoutlines=MB_outlines_xz,ROIname="MB outline")
plotSynapseGroups([PAMPAMpresynapses,PAMPAMpostsynapses],title, coordinates=["x","y"],ROIoutlines=MB_outlines_xy,ROIname="MB outline")

In [ ]:
print("Printing ROIs with PAM-PAM presynapses")
print(sorted(filter(None, PAMPAMpresynapses["roi_pre"].unique())))
print(sorted(filter(None, PAMPAMpostsynapses["roi_pre"].unique())))
print("\nPrinting ROIs with PAM-PAM postsynapses")
print(sorted(filter(None, PAMPAMpresynapses["roi_post"].unique())))
print(sorted(filter(None, PAMPAMpostsynapses["roi_post"].unique())))


In [ ]:
noneRoiPresynapses = PAMPAMpresynapses[all_PAM_PAM_presynapses["roi_post"].isnull()]
noneRoiPostsynapses = PAMPAMpostsynapses[all_PAM_PAM_postsynapses["roi_post"].isnull()]

### analyzing MB and non-MB synapses

In [ ]:
## get PAM neurons 
neuron_criteria = NC(status='Traced', type="^PAM.*",regex=True)

In [ ]:
## get synapses within MB
MB_rois=["a'L(R)","aL(R)","b'L(R)","bL(R)","gL(R)","CA(L)","a'L(L)","aL(L)","b'L(L)","bL(L)","gL(L)", "CA(R)", "PED(R)"]
MBpresynapses_criteria = SC(type='pre', primary_only=True,rois=MB_rois)
MBpostsynapses_criteria = SC(type='post', primary_only=True,rois=MB_rois)
#MB_PAM_PAM_presynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, MBpresynapses_criteria)
#MB_PAM_PAM_postsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, MBpostsynapses_criteria)
MB_PAM_PAM_presynapses = loadPickle("MB_PAM_PAM_presynapses")
MB_PAM_PAM_postsynapses = loadPickle("MB_PAM_PAM_postsynapses")

In [ ]:
## get synapses outside MB
### previously, these ROIs have been found to be the ROIs outside the MB that have PAM-PAM connections
non_MB_rois=["CRE(L)", "CRE(R)", "EB", "LAL(R)", "SIP(L)", "SIP(R)", "SLP(R)", "SMP(L)", "SMP(R)", "LAL(L)"]
nonMBpresynapses_criteria = SC(type='pre', primary_only=True,rois=non_MB_rois)
nonMBpostsynapses_criteria = SC(type='post', primary_only=True,rois=non_MB_rois)
#nonMB_PAM_PAM_presynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, nonMBpresynapses_criteria)
#nonMB_PAM_PAM_postsynapses = fetch_synapse_connections(neuron_criteria, neuron_criteria, nonMBpostsynapses_criteria)
nonMB_PAM_PAM_presynapses = loadPickle("nonMB_PAM_PAM_presynapses")
nonMB_PAM_PAM_postsynapses = loadPickle("nonMB_PAM_PAM_postsynapses")

In [ ]:
MBpresynapses = filterSynapseConfidence(MB_PAM_PAM_presynapses,mode="pre")
MBpostsynapses = filterSynapseConfidence(MB_PAM_PAM_postsynapses,mode="post")
nonMBpresynapses = filterSynapseConfidence(nonMB_PAM_PAM_presynapses,mode="pre")
nonMBpostsynapses = filterSynapseConfidence(nonMB_PAM_PAM_postsynapses,mode="post")

In [ ]:
print(MBpresynapses.index.size)
print(MBpostsynapses.index.size)
print(nonMBpresynapses.index.size)
print(nonMBpostsynapses.index.size)


In [ ]:
filteredMBpre=filterSynapseConfidence(MBpresynapses,mode="pre",percentile=90)
filterednonMBpre=filterSynapseConfidence(nonMBpresynapses,mode="pre",percentile=90)

In [ ]:
print(filteredMBpre.index.size)
print(filterednonMBpre.index.size)
# todo investigate these non-MB-synapses coordinates

##### Visualization of PAM-PAM synapses inside/outside MB

In [ ]:
# Plot the synapse positions in a 2D projection
## blue synapses are MB synapses, red are outside the MB
title = "PAM-PAM synapses (red: MB, blue: non-MB, green: ROI 'None')"

plotSynapseGroups([MBpresynapses,nonMBpresynapses,noneRoiPresynapses],title,ROIoutlines=MB_outlines_xz,ROIname="MB outline")
plotSynapseGroups([MBpresynapses,nonMBpresynapses,noneRoiPresynapses],title, coordinates=["x","y"],ROIoutlines=MB_outlines_xy,ROIname="MB outline")

### analysis of synapse details

In [ ]:
all_PAM_PAM_presynapses

In [ ]:
#all_neurons, roi_count = neu.fetch_neurons(NC(status='Traced',type="^.*",regex=True)) 
all_neurons = loadPickle("all_neurons")

In [ ]:
from utils import collapseNeuronNames, classifySynapseConnectivity

In [ ]:
mergedPPpresynapses = neu.merge_neuron_properties(all_neurons,all_PAM_PAM_presynapses,properties=["type", "instance"])
mergedPPpostynapses = neu.merge_neuron_properties(all_neurons,all_PAM_PAM_postsynapses,properties=["type", "instance"])

In [ ]:
mergedPPpresynapses

In [ ]:
from utils import filterSynapseConfidence

In [ ]:
title = "PAM-PAM Synapse Types"
mergedPPpresynapses = classifySynapseConnectivity(mergedPPpresynapses)
mergedPPpostynapses = classifySynapseConnectivity(mergedPPpostynapses)
plotSynapseClassification(mergedPPpresynapses,title=title,ROIoutlines=MB_outlines_xz,ROIname="MB outline")
plotSynapseClassification(mergedPPpresynapses, title=title, coordinates=["x","y"],ROIoutlines=MB_outlines_xy,ROIname="MB outline")

In [ ]:
merged = neu.merge_neuron_properties(all_neurons,nonMBpostsynapses,properties=["type", "instance"])
classified = classifySynapseConnectivity(merged)
filtered=classified[classified["type_post"]=="PAM01_a"]
plotSynapseClassification(filtered,title=title,ROIoutlines=MB_outlines_xz,ROIname="MB outline")
plotSynapseClassification(filtered, title=title, coordinates=["x","y"],ROIoutlines=MB_outlines_xy,ROIname="MB outline")

In [ ]:
filtered

In [ ]:
## fetch all synapses that involve PAM,
## postsynapses are 1:1 identical to presynapses, both have n=150049 
pam_criteria = NC(status='Traced', type="^PAM.*",regex=True)
all_criteria = NC(status='Traced', type="^.*",regex=True)

allPAMpresynapses_criteria = SC(type='pre', primary_only=True)
#allPAMpostsynapses_criteria = SC(type='post', primary_only=True)
#all_PAM_All_presynapses = fetch_synapse_connections(pam_criteria, all_criteria, allPAMpresynapses_criteria,batch_size=10000)
#all_PAM_All_postsynapses = fetch_synapse_connections(all_criteria, pam_criteria, allPAMpostsynapses_criteria,batch_size=10000)
all_PAM_All_presynapses = loadPickle("all_PAM_All_presynapses")
all_PAM_All_postsynapses = loadPickle("all_PAM_All_postsynapses")

In [ ]:
PAMpresynapses = filterSynapseConfidence(all_PAM_All_presynapses,mode="pre",percentile=10) 
mergedPAMpresynapses = neu.merge_neuron_properties(all_neurons,PAMpresynapses,properties=["type", "instance"])
#mergedPAMpostynapses = neu.merge_neuron_properties(all_neurons,allPAMpostsynapses,properties=["type", "instance"])

mergedPAMpresynapses = classifySynapseConnectivity(mergedPAMpresynapses)
#mergedPAMpostynapses = classify_connections(mergedPAMpostynapses)

In [ ]:
title="PAM Presynapses Classification"
decimated_connections = mergedPAMpresynapses.sample(n=10000)
plotSynapseClassification(decimated_connections, title=title, coordinates=["x","z"],ROIoutlines=MB_outlines_xz,ROIname="MB")
plotSynapseClassification(decimated_connections, title=title, coordinates=["x","y"],ROIoutlines=MB_outlines_xy,ROIname="MB")


In [ ]:
import importlib
importlib.reload(utils)
from utils import plotPAMTypePresynapses


In [ ]:
plotPAMTypePresynapses(mergedPAMpresynapses,pamType="PAM01",roi_outlines=[MB_outlines_xz,MB_outlines_xy],roiName="MB")

# Neurotransmitter

In [ ]:
neurotransmitters = pd.read_feather("pickles/Hemibrain Neurotransmitters.feather")



In [ ]:
neurotransmitters.sort_values("nts_8.dopamine")

In [ ]:
from utils import getROIs

MBroisRegex = getROIs("MB","regexOr")
nonMBroisRegex = getROIs("nonMB","regexOr")
## regex search term for all ROIs that we know to contain PAM neurons
allROIsRegex = MBroisRegex + "|" + nonMBroisRegex
allROIsRegex

In [ ]:
neurotransmitters_filtered = neurotransmitters[neurotransmitters['roi'].str.contains(allROIsRegex,regex=True)]
neurotransmitters_filtered


In [ ]:
PAMPAMpresynapses

In [ ]:
# Assuming PAM_PAM_connections is already defined somewhere in the notebook
merged_df = pd.merge(neurotransmitters, PAMPAMpostsynapses, left_on=['sv', 'body'], right_on=['bodyId_pre', 'bodyId_post'])
merged_df


In [ ]:
all_PAM_All_postsynapses 
all_PAM_All_postynapsesMerged = neu.merge_neuron_properties(all_neurons,all_PAM_All_postsynapses,properties=["type", "instance"])
all_PAM_All_postynapsesMerged

In [ ]:
all_PAM_All_postynapsesMergedFiltered = all_PAM_All_postynapsesMerged[all_PAM_All_postynapsesMerged['type_post'].str.contains("PAM", regex=True)]
all_PAM_All_postynapsesMergedFiltered

In [ ]:
all_PAM_All_presynapsesMergedFiltered = all_PAM_All_presynapsesMergedFiltered[all_PAM_All_presynapsesMerged['type_pre'].str.contains("KC", regex=True)]